In [ ]:
# History
# Name      Date          Description                   Issue
# Bofan     Nov 20        upload and load dataset.      file too large to load and causing crash
# Emanuel   Dec 1         removing reviews stopwords    

## E

In [56]:
# B
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import cosine_similarity

filename = 'prep.csv'
df = pd.read_csv(filename, index_col='Unnamed: 0')
df.head()

,overall,asin,reviewText,summary,unixReviewTime
452,4,7245456275,These bands are high quality and very easy to ...,Good supplement to weights,1301011200
453,4,7245456275,This is a great addition to your work out and ...,Good for the price,1301011200
454,5,7245456275,The lighter 4 pound resistance band is a perfe...,4 LBS resistance bands,1288569600
455,5,7245456275,BMP has made some new improvements on their pr...,10 LBS band review,1288569600
456,5,7245456275,I run a small fitness company and we use these...,5 LBS band review,1288569600


In [57]:
df = df.sample(n = 1000)

In [58]:
# Functions
def remove_punct(review):
    tokenizer = RegexpTokenizer(r'\w+')
    review = tokenizer.tokenize(review)
    review = " ".join(review)
    return review


In [59]:
# E 
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop = stopwords.words('english')
df = df.dropna()
df_asin = df[['asin', 'reviewText']].copy()
df_asin['reviewText'] = df_asin['reviewText'].str.lower()
df_asin['reviewText'] = df_asin['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_asin['reviewText'] = df_asin['reviewText'].apply(func = remove_punct)
#df_asin['reviewText'] = df_asin['reviewText'].apply(word_tokenize)
df_asin

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emanuel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/emanuel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,asin,reviewText
52618,B000J05EYI,i m always hunting around house ear plugs use ...
33653,B000NORVHI,great
55025,B0017VXC3I,money great buy
66817,B0016LJWEW,super easy install use
44901,B00J0S018E,needed
...,...,...
6806,B016AQXG58,skates amazing caution toe stops cannot adjust...
29320,B0068I2E3E,came fast great quality pleased good beginner ...
66777,B00NI4NUFC,wide great grip
63537,B006YVT0R8,great scope put ar completely sighted 5 shots ...


In [60]:
# E
dict_asin = df_asin.set_index('asin').T.to_dict('list')
dict_asin

/var/folders/cg/52ls_8wn3bx7qsf2ppb2s6300000gn/T/ipykernel_41125/3077481213.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dict_asin = df_asin.set_index('asin').T.to_dict('list')


{'B000J05EYI': ['i m always hunting around house ear plugs use power equipment around house wash re use end different places got set recent mri amazed much better standard foam ones i d using bought whole box never chase around finding set'],
 'B000NORVHI': ['great'],
 'B0017VXC3I': ['gift friend helped back pain when nothing else worked'],
 'B0016LJWEW': ['super easy install use'],
 'B00J0S018E': ['needed'],
 'B00KSMVACW': ['love it i m lifeguard water park can t leave post came handy fill least 3 4 gallon bladder lasts whole day'],
 'B01GK97UX4': ['works great'],
 'B00QR3LPXK': ['functions compass really vide solid construction read illuminated rose difficulties weak point would glass case careless handling'],
 'B000SZKHNC': ['birthday gift love enjoy swimming running looks good home last awhile'],
 'B00IEF8N6Q': ['seems little large first works great take every ride'],
 'B010943M9C': ['awesome product'],
 'B014Q57LG0': ['love it'],
 'B016D2OTSW': ['excellent quality workmanship perf

In [76]:
# E
from itertools import chain
bow = []
for arr in df_asin['reviewText']:
    bow.append(arr)
bow = list(chain.from_iterable(bow))
bow = set(bow)
bow

{' ',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = []
for s in df_asin['reviewText']:
    corpus.append(s)
corpus

['i m always hunting around house ear plugs use power equipment around house wash re use end different places got set recent mri amazed much better standard foam ones i d using bought whole box never chase around finding set',
 'great',
 'money great buy',
 'super easy install use',
 'needed',
 'love it i m lifeguard water park can t leave post came handy fill least 3 4 gallon bladder lasts whole day',
 'works great',
 'functions compass really vide solid construction read illuminated rose difficulties weak point would glass case careless handling',
 'birthday gift love enjoy swimming running looks good home last awhile',
 'seems little large first works great take every ride',
 'awesome product',
 'thing ba already put use arrived faster estimated color even better pic',
 'excellent quality workmanship perfect fit take professional holster collector unwanted title best bang yuk yuk buck',
 'great product fantastic price quick service',
 'really happy bottles good size good packing lov

In [73]:
vectorizer = CountVectorizer()
Xm = vectorizer.fit_transform(df_asin['reviewText'])
features = vectorizer.get_feature_names_out()
Xarr = Xm.toarray()

df_vect = pd.DataFrame(data = Xarr, columns = features, index = df_asin['asin'])
df_vect

,00,10,100,1000,10000000000,1000lm,1090,11,12,1200,...,z2,zapper,zero,zip,zipped,zipper,zippered,zippers,zipping,zone
asin,,,,,,,,,,,,,,,,,,,,,
B000J05EYI,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B000NORVHI,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0017VXC3I,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0016LJWEW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00J0S018E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B016AQXG58,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0068I2E3E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00NI4NUFC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
# Cosine Similarity
co_sim = cosine_similarity(Xm)
co_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.08006408],
       [0.        , 1.        , 0.57735027, ..., 0.57735027, 0.25819889,
        0.        ],
       [0.        , 0.57735027, 1.        , ..., 0.33333333, 0.2981424 ,
        0.16012815],
       ...,
       [0.        , 0.57735027, 0.33333333, ..., 1.        , 0.1490712 ,
        0.        ],
       [0.        , 0.25819889, 0.2981424 , ..., 0.1490712 , 1.        ,
        0.07161149],
       [0.08006408, 0.        , 0.16012815, ..., 0.        , 0.07161149,
        1.        ]])

In [ ]:
# B

# 2. ML architecture

In [ ]:
  # input                                                                                                                                                                                     # output  
  # ################                                                                                                                                                                          ##################################
  # (clients search# --------> word encoding (replace each text in every review) --------> dense embedding -------->       MLP of NN   -------->  choose a/ mult classifier   -------->       # (rank list of product (sentiment/ classification??)
  # key sentence)###                                                                      n-dimentions vec             as input for the NN          compare acurrancy                         # which is most relevant to search corpus)
  # existing review#                                                                                                                                 choose winning classfier/cluster         # !!!!!!relevent review's prodect ids !!!!
  # ################                                                                                                                                   for prediction model                   ##################################  
  # 
  #                                                                                                                                   -------->    choose a algorthm for sentiment
  # 

In [ ]:
# dense embedding  refer to link: https://www.tensorflow.org/text/guide/word_embeddings


#!!!! design a DN cluster that input one review and output all reviews in that group and sort by distance